# 1 RHESSI Solar Flares

https://hesperia.gsfc.nasa.gov/rhessi3/data-access/rhessi-data/flare-list/index.html

http://sprg.ssl.berkeley.edu/~jimm/hessi/hsi_flare_list.html

http://sprg.ssl.berkeley.edu/%7Etohban/browser/?show=grth1+qlpcr+qlpds+qlpg9+qlifs+qli01+qli02+qli03+qli04+qli05+qli06+qlids&date=20020212&time=213530&bar=1
GOES classes: https://svs.gsfc.nasa.gov/10109

In [ ]:
import numpy as np
import scipp as sc

### 1.1 Load Flare List

In [ ]:
flares = sc.io.open_hdf5('data/hessi_flares.h5')
flares

The data (`flares.data`) contains a weight for each flare.
Initially, all weights are 1.

The most important metadata items are:
- `start_time`, `end_time`: Time interval of flare.
- `peak_time`: Date and time of the highest flux.
- `x`, `y`: Position in the image seen by RHESSI.
- `min_energy`, `max_energy`: Energy band that a flare was observed in. Bands do not overlap.
- `non_solar`: The event was flagged as not coming from the Sun.

### 1.2 Inspect the Loaded Data

Begin by inspecting the data array to gain a basic understanding of the data.
This task is open-ended, and you can continue when you feel confident that you know what `flares` contains.

Possible actions:
- Use [scipp.show](https://scipp.github.io/generated/functions/scipp.show.html) and [scipp.table](https://scipp.github.io/generated/functions/scipp.table.html) in addition to the HTML output of the cell above.
- Extract individual coordinates and attributes using `flares.coords['<name>']` and `flares.attrs['<name>']`.
- Plot the data. To get meaningful plots, you can construct auxiliary data arrays and plot them using

```python
sc.plot(sc.DataArray(da.meta['<what_goes_on_y_axis>'],
                     coords={'<name>': '<what_goes_on_x_axis>'}))
```

You can answer the following questions or find your own.
- How many flares are there in the dataset?
- How many flares are flagged as 'non_solar'?
- What is the time range of the data?

**Tip**: Use [scipp.sum](https://scipp.github.io/generated/functions/scipp.sum.html), [scipp.max](https://scipp.github.io/generated/functions/scipp.max.html), and [scipp.min](https://scipp.github.io/generated/functions/scipp.min.html) with the coordinates and attributes of `flares`.

In [ ]:
# number of flares:
flares.sizes['flare']

In [ ]:
# number flagged as non-solar:
flares.attrs['non_solar'].sum()

In [ ]:
# time range:
flares.coords['start_time'].min(), flares.coords['end_time'].max()

### 1.3 Compute Duration

Calculate the duration of flares as `end_time - start_time` and store the result as a new coordinate in `flares`.
Remember that `flares.coords` functions like a Python `dict`.

In [ ]:
# simple
duration = flares.coords['end_time'] - flares.coords['start_time']
flares.coords['duration'] = duration


# advanced
# def compute_duration(start_time, end_time):
#     return end_time - start_time
#
#
# flares.transform_coords('duration', {'duration': compute_duration})

### 1.4 Create Masks

Some events in the input data did not originate from the Sun.

There are two options for handling those events, removing them or masking them.
You can choose a solution, but the descriptions guide you through masking.
This is a method for removing events non-destructively, that is, the masks can be removed later to get the full event list back to determine the impact of the masks.

First, store `'non_solar'` as a mask and remove it from the attributes.
Use `flares.attrs` and `flares.masks` which are `dict`-like objects (similar to `flares.coords`).

In [ ]:
flares.masks['non_solar'] = flares.attrs.pop('non_solar')

Second, there are some flares where it was impossible to determine their position.
Those are stored with `x == y == 0` and need to be removed, as well.

Construct a boolean variable by comparing `flares.coords['<x_or_y>']` to 0.
Note that `x` and `y` have unit 'asec'.
This means that you have to construct a '0' with the same unit which can be done using `0 * sc.Unit('asec')`.

Finally, combine the variables for `x` and `y` using `mask_x | mask_y` and store the result as a new mask in `flares`.

In [ ]:
flares.masks['unknown_position'] = (
    (flares.coords['x'] == 0 * sc.Unit('asec')) |
    (flares.coords['y'] == 0 * sc.Unit('asec')))

## 2 Spatial Distribution

### 2.1 Bin by x and y

Plot the spatial distribution of flares.

Plotting the event list `flares` would yield a scatter plot which is not particularly useful.
A better approach is computing and plotting the density as a function of `x` and `y`.
This is commonly done by histogramming the events.
But in scipp, there is a better alternative: binning.

Scipp's 'binned data' is similar to a histogram, except that the individual events are preserved.
They are simply collected into bins as defined by bin-edge coordinates.

Define bin-edges for `x` and `y`, use [scipp.bin](https://scipp.github.io/generated/functions/scipp.bin.html) to create binned data from `flares`, and plot the result.

**Tip** 
- Use [scipp.arange](https://scipp.github.io/generated/functions/scipp.arange.html) or [scipp.linspace](https://scipp.github.io/generated/functions/scipp.linspace.html) to construct the edges. Make sure to use the correct unit!
- Use `sc.bin(flares, edges=[<edge_x>, <edge_y>])`.
- Turn your binned data into a histogram before plotting using `<binned>.bins.sum()`.

In [ ]:
spatial = sc.bin(flares,
                 edges=[
                     sc.linspace('y', -1200, 1200, 100, unit='asec'),
                     sc.linspace('x', -1200, 1200, 100, unit='asec')
                 ])

In [ ]:
spatial.bins.sum().plot(aspect='equal',
                        norm='log',
                        labels={
                            'x': 'x [asec]',
                            'y': 'y [asec]'
                        })

### 2.2 Remove Outliers

The plot shows a lot of outliers.
These are caused by the RHESSI's electronics or analysis software glitching out and assigning bad positions the flares.
This includes the circular shape event though it looks deceptively sun-like.

The flare list does not include enough information to exclude all such bad positions.
But the instrument can only detect x-rays for $x \in [-1000~\text{asec}, 1000~\text{asec}]$ and $y \in [-600~\text{asec}, 600~\text{asec}]$.
So to first order, all events outside that range should be removed.

Instead of using a mask as before, use binning this time.
Create new bin-edges for `x` and `y` with the proper limits, this will remove all events outside the valid range.
Bin the data with the new edges and plot the result.
(You can either bin the original `flares` data array or apply new bins to the previously binned array; both via [scipp.bin](https://scipp.github.io/generated/functions/scipp.bin.html)).

In [ ]:
# n bins such that bins are square
spatial = sc.bin(flares,
                 edges=[
                     sc.linspace('y', -600, 600, 90, unit='asec'),
                     sc.linspace('x', -1000, 1000, 150, unit='asec')
                 ])

In [ ]:
spatial.bins.sum().plot(aspect='equal',
                        norm='log',
                        labels={
                            'x': 'x [asec]',
                            'y': 'y [asec]'
                        })

### 2.3 Correct for Detector Efficiency

In this tutorial, we assume that the instrument consists of a 3x3 grid of detectors which each record x-rays from distinct directions.
(The reality is more complicated, of course. See the [wiki](https://sprg.ssl.berkeley.edu/~tohban/wiki/index.php/Category:RHESSI) for more information.)
Furthermore, the tutorial data has been manipulated to simulate different efficiencies of the individual detectors.

The efficiency is available as an attribute:

In [ ]:
efficiency = flares.attrs['detector_efficiency'].value
efficiency

Normalize the data by dividing by `efficiency`.
You first need to bin into the correct bins as defined by the coordinates of `efficiency`.

In [ ]:
spatial = sc.bin(spatial, edges=[efficiency.coords['x'], efficiency.coords['y']])
corrected = spatial / efficiency

You can plot the corrected data as before.
But it makes sense to return to smaller bins in order to resolve the distribution properly.
This can be done using [scipp.bin](https://scipp.github.io/generated/functions/scipp.bin.html) with finer edges because the data was only binned and not histogrammed.

In [ ]:
# n bins such that bins are square
spatial = sc.bin(spatial,
                 edges=[
                     sc.linspace('y', -600, 600, 90, unit='asec'),
                     sc.linspace('x', -1000, 1000, 150, unit='asec')
                 ])

In [ ]:
spatial.bins.sum().plot(aspect='equal',
                        norm='log',
                        labels={
                            'x': 'x [asec]',
                            'y': 'y [asec]'
                        })

## 3 Time Series

Now, we want to look at the distribution of flares over time.
The temporal distribution can be obtained like the spatial using binning.

Select one of the time coordinates (e.g. 'peak_time') and construct appropriate bin-edges using [scipp.arange](https://scipp.github.io/generated/functions/scipp.arange.html) that cover the whole range.
Bin the data by those edges and plot the result.

**Important** Use the pre-binned data from the previous task in order to include the detector normalization.

**Tip**
- The time coordinates are event-coordinates, that is, they are not defined at the top level of the binned data (i.e. per bin) but inside of the bins (i.e. per event). You can access them using `<binned>.bins.coords['preak_time']`.
- You can add binning by time using `sc.bin(<binned>, edges=[<time_edges>])` and produce a three-dimensional array.
  But here, we are more interested in a one-dimensional distribution which can be obtained using `sc.bin(<binned>, edges=[<time_edges>], erase=['x', 'y'])`.

In [ ]:
time = spatial.bins.coords['peak_time']
min_time = time.min().value
max_time = time.max().value
step = (max_time - min_time) / 200
by_time = sc.bin(spatial, edges=[sc.arange('peak_time', min_time, max_time, step, unit=time.bins.unit)], erase=['x', 'y'])

In [ ]:
by_time.bins.sum().plot(ylabel=r'$n_{\mathsf{flares}}$')

### 3.1 Flare Durations

Another interesting quantity to look at is the duration of flares.
The duration was computed earlier and should already be stored as an event coordinate.

Plot the duration as a function of time.

**Tip**
- Construct a new data array using the previous result: `duration = <binned_by_time>.copy()`.
  And assign new data using `duration.bins.data = <duration_data>`.
- Previously, we used `.bins.sum()` to make histograms because the data was given as counts.
  Now the data is seconds which should be averaged instead of summed.
  So use `.bins.mean()` instead.

In [ ]:
duration = by_time.copy()
duration.name = 'duration'
duration.bins.data = duration.bins.coords.pop('duration')
duration

In [ ]:
duration.bins.mean().plot()

## 4 Energy Bands

The flares were recorded in several non-overlapping energy bands.
Those are identified by the 'min_energy' and 'max_energy' attributes in `flares`.
Since the bands do not overlap, it is sufficient to label them with 'min_energy'.

In this section, we want to split the temporal distribution from above into the different energy bands.
First, find all minimum energies and construct a `sc.Variable` containing them.

**Tip**
- Use `flares.attrs['min_energy']` and either plot it or use [numpy.unique](https://numpy.org/doc/stable/reference/generated/numpy.unique.html) to find out what values it can take.
- Use [scipp.array](https://scipp.github.io/generated/functions/scipp.array.html) to make the variable.
    - Choose a new dimension label, e.g. 'min_energy'
    - Don't forget the unit!

In [ ]:
min_energies = sc.array(dims=['min_energy'],
                        values=np.unique(flares.attrs['min_energy'].values),
                        unit=flares.attrs['min_energy'].unit)
min_energies

Bin the temporal distribution by 'min_energy' to obtain two-dimensional data.
Use the `groups` argument of [scipp.bin](https://scipp.github.io/generated/functions/scipp.bin.html) instead of edges this time because every event has exactly one of a set of possible energies instead of a value in a range.

In [ ]:
# Optional: Drop lowest energy band because those events are not confirmed flares.
by_time_and_energy = sc.bin(by_time, groups=[min_energies[1:]])

Plot the result.
A 2D plot is not very useful here, so split the data by 'min_energy' and either plot each energy in a separate plot or combine them into a dictionary and plot that:
```python
lines = {'<name>': <data_array>}
sc.plot(lines)
```

In [ ]:
lines = {
    f"min_energy={by_time_and_energy['min_energy', i].attrs['min_energy'].value}":
    by_time_and_energy['min_energy', i].bins.sum()
    for i in range(by_time_and_energy.sizes['min_energy'])
}
sc.plot(lines, ylabel=r'$n_{\mathsf{flares}}$')